# XGBoost On Boston Housing

## 1.Import

In [5]:
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import seaborn as sns

## 2.Load Data

In [6]:
# Resolve project paths and organize outputs
PROJECT_ROOT = Path.cwd()
if not (PROJECT_ROOT / "datasets").exists() and (PROJECT_ROOT.parent / "datasets").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent
if not (PROJECT_ROOT / "datasets").exists():
    raise FileNotFoundError("Could not locate datasets folder. Run from project root or src directory.")

DATA_DIR = PROJECT_ROOT / "datasets" / "boston-housing"
OUTPUT_DIR = PROJECT_ROOT / "outputs"
PLOTS_DIR = OUTPUT_DIR / "plots"
MODELS_DIR = OUTPUT_DIR / "models"
PREDICTIONS_DIR = OUTPUT_DIR / "predictions"
for path in [OUTPUT_DIR, PLOTS_DIR, MODELS_DIR, PREDICTIONS_DIR]:
    path.mkdir(parents=True, exist_ok=True)

print(f"Project root: {PROJECT_ROOT}")
print(f"Outputs directory: {OUTPUT_DIR}")

Project root: d:\学习资料\数据挖掘\data-mining
Outputs directory: d:\学习资料\数据挖掘\data-mining\outputs


In [7]:
data_path = DATA_DIR / "HousingDataPreprocessed.csv"
df = pd.read_csv(data_path, sep=",")
features_list = list(df.columns)
features_list.remove('MEDV')
X = df[features_list]
Y = df['MEDV']

In [8]:
X['CHAS'] = X['CHAS'].astype('bool')

C:\Users\MSI_NB\AppData\Local\Temp\ipykernel_16868\578400245.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['CHAS'] = X['CHAS'].astype('bool')


## 3. 5 Fold Cross Verification

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,)

In [10]:
'''
n_jobs = -1
param_grid = {
    'n_estimators': [100, 200, 300, 500],
     'max_depth': [3, 4, 5, 6, 7, 8],
    'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3, 0.4],
    'reg_alpha': [0, 0.01, 0.1, 1, 10],
    'reg_lambda': [0.1, 1, 5, 10, 50]
}

base_model = xgb.XGBRegressor(
    random_state=20251204,
    n_jobs=n_jobs,
    verbosity=0
)
        

search = GridSearchCV(
    estimator=base_model,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=5,  # 5折交叉验证
    verbose=1,
    n_jobs=n_jobs,
    return_train_score=True
            )
        
search.fit(X_train, Y_train)
        
# 保存结果
model = search.best_estimator_
best_params = search.best_params_
cv_results = pd.DataFrame(search.cv_results_)
        
print("\n" + "=" * 60)
print("超参数调优完成！")
print("=" * 60)
print(f"最佳参数: {best_params}")
print(f"最佳交叉验证分数 (MSE): {-search.best_score_:.4f}")
'''

'\nn_jobs = -1\nparam_grid = {\n    \'n_estimators\': [100, 200, 300, 500],\n     \'max_depth\': [3, 4, 5, 6, 7, 8],\n    \'learning_rate\': [0.01, 0.05, 0.1, 0.2, 0.3],\n    \'subsample\': [0.6, 0.7, 0.8, 0.9, 1.0],\n    \'colsample_bytree\': [0.6, 0.7, 0.8, 0.9, 1.0],\n    \'gamma\': [0, 0.1, 0.2, 0.3, 0.4],\n    \'reg_alpha\': [0, 0.01, 0.1, 1, 10],\n    \'reg_lambda\': [0.1, 1, 5, 10, 50]\n}\n\nbase_model = xgb.XGBRegressor(\n    random_state=20251204,\n    n_jobs=n_jobs,\n    verbosity=0\n)\n\n\nsearch = GridSearchCV(\n    estimator=base_model,\n    param_grid=param_grid,\n    scoring=\'neg_mean_squared_error\',\n    cv=5,  # 5折交叉验证\n    verbose=1,\n    n_jobs=n_jobs,\n    return_train_score=True\n            )\n\nsearch.fit(X_train, Y_train)\n\n# 保存结果\nmodel = search.best_estimator_\nbest_params = search.best_params_\ncv_results = pd.DataFrame(search.cv_results_)\n\nprint("\n" + "=" * 60)\nprint("超参数调优完成！")\nprint("=" * 60)\nprint(f"最佳参数: {best_params}")\nprint(f"最佳交叉验证分数 (MSE): 

In [11]:
!pip install scikit-optimize -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

search_spaces = {
            'n_estimators': Integer(1100, 1400),
            'max_depth': Integer(2, 5),
            'subsample': Real(0.5, 0.7),
            'colsample_bytree': Real(0.65, 0.85),
            'gamma': Real(0, 5),
            'reg_alpha': Real(1e-12, .1, 'log-uniform'),
            'reg_lambda': Real(1e-12, .1, 'log-uniform'),

    }
        

base_model = xgb.XGBRegressor(
    random_state=20251204,
    n_jobs=-1,
    verbosity=1,
    learning_rate=0.01,
    min_child_weight = 1,
)
    
bayes_search = BayesSearchCV(
    estimator=base_model,
    search_spaces=search_spaces,
    n_iter=240,
    cv=5,  # 5折交叉验证
    scoring='neg_mean_squared_error',
    random_state=20251204,
    n_jobs=-1,
    verbose=1,
)
        
# 执行搜索
bayes_search.fit(X_train, Y_train)


# 保存结果
model = bayes_search.best_estimator_
best_params = bayes_search.best_params_
search_results = bayes_search

print("\n" + "=" * 60)
print("贝叶斯优化结果")
print("=" * 60)
print(f"最佳参数: {best_params}")
print(f"最佳交叉验证分数 (MSE): {-bayes_search.best_score_:.4f}")


SyntaxError: invalid syntax. Perhaps you forgot a comma? (3412755809.py, line 19)

最佳参数: OrderedDict([('colsample_bytree', 0.8477197877691871), ('gamma', 2.963020908406329), ('learning_rate', 0.01246022881782795), ('max_depth', 3), ('min_child_weight', 1), ('n_estimators', 995), ('reg_alpha', 1e-08), ('reg_lambda', 1e-08), ('subsample', 0.678350374817612)])
最佳交叉验证分数 (MSE): 9.6028

最佳参数: OrderedDict([('colsample_bytree', 0.681285771749176), ('gamma', 0.0), ('learning_rate', 0.009079813094509085), ('max_depth', 5), ('min_child_weight', 1), ('n_estimators', 1341), ('reg_alpha', 0.1), ('reg_lambda', 1e-12), ('subsample', 0.5)])
最佳交叉验证分数 (MSE): 8.4320